In [ ]:
############################################################################################################
# 2024/03/28
# convert a directory of converted ioda.nc4 GPSRO n files to pandas dataframe

import pandas as pd
from netCDF4 import Dataset
import sys
from GetInputFileListEnumerate import *
import subprocess

ioda_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO_final/Y2022/M01'
saved_dataframe_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO_final/Y2022/GPSRO.SPNASA.Y2022.M01.ioda_nc4.csv'
plot_filename = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO_final/figures/GPSRO.SPNASA.Y2022.M01.ioda_nc4.p1.counts.png'


####################################
#def iodanc4_to_dataframe(files_ioda):
####################################
q = []
df_all = pd.DataFrame(q, columns = ['dateTime', 'latitude', 'longitude', 'bendingAngle','impp',
                                    'satid','seq', 'satConstRO', 'satTransId','geoidU', 'refr',
                                    'filename','filepath'])



# 10/24/2023
# GetInputFileListEnumerate() ----------------------------------------------------------------
import os
import sys
    # path = '/discover/nobackup/sicohen/Data/gmao/bufr/AMSUA/Y2020/'
    # rootdir = sys.argv[1]
    # 'filelist' with be the list of bufr files that is returned (to then be used as input for the inventory tool)
    # loop over directory and all it's subdirectories, list whats in there but dont print everything, return only bufr files account for files name with extensions '.bufr' and '.bufr_d'
def GetInputFileListEnumerate(rootdir):#,suffix):
    filelist = []
    for subdir, dirs, files in os.walk(rootdir):
        print(f"{dirs}")
        print(f"----{subdir}")
        for idx, file in enumerate(files):
            if file.endswith("ioda.nc4"): # or file.endswith(".bufr_d"):
                print (f"--------- {os.path.join(subdir, file)}")
                filepath = subdir + os.sep + file
                filelist.append(filepath)
    return(sorted(filelist))    #print(sorted(filelist))

# list of files 
files_ioda = GetInputFileListEnumerate(ioda_path)


j=0
for i in files_ioda:
    j+=1
    print(f'Converting (.nc4 to pandas Dataframe) file {j} out of {len(files_ioda)}: {i}')
    data = Dataset(i, mode='r')
    df = pd.DataFrame(
        {
            'dateTime': data.groups['MetaData'].variables['dateTime'][:], 
            'latitude': data.groups['MetaData'].variables['latitude'][:],
            'longitude': data.groups['MetaData'].variables['longitude'][:],
            'bendingAngle': data.groups['ObsValue'].variables['bendingAngle'][:],
            'impp': data.groups['MetaData'].variables['impactParameterRO'][:],
            'satid': data.groups['MetaData'].variables['satelliteIdentifier'][:],
            'seq': data.groups['MetaData'].variables['sequenceNumber'][:],
            'satConstRO': data.groups['MetaData'].variables['satelliteConstellationRO'][:],
            'satTransId': data.groups['MetaData'].variables['satelliteTransmitterId'][:],
            'geoidU': data.groups['MetaData'].variables['geoidUndulation'][:],
            'refr': data.groups['ObsValue'].variables['atmosphericRefractivity'][:],
            'filename':i.split("/")[-1],
            'filepath':i,
            'filedate':i.split(".")[1],
            'filecycle':i.split(".")[2],
            'filedatetime':i.split(".")[1] + i.split(".")[2],
            'DATE': pd.to_datetime(data.groups['MetaData'].variables['dateTime'][:],unit='s') 
                    #pd.to_datetime(df['dateTime'],unit='s')
        }
    )
    #print(df)
    print(i)
    df_all = pd.concat([df_all,df])
    print(f' Progress: {round(j/len(files_ioda)*100)}%')
    #return df_all
####################################

df_all.sort_values(by=['filename','DATE'])
df=df_all


# 
#df['dateTime'] = pd.to_datetime(df['dateTime'], format='s')
#df['DATE'] = pd.to_datetime(df['dateTime'],unit='s')

# Merge the Bufr SAID codes ~ add names for satid's
############################################################
BufrCodes = pd.read_csv('/discover/nobackup/sicohen/Workshop/SicInventoryTools/dependencies/bufr-code-table-C-sinv-merge.csv')
BufrCodes['SAID'] = BufrCodes['SAID'].astype(float).astype(int)
BufrCodes['kx'] = BufrCodes['SAID'].abs().astype(int)
df['kx'] = df['satid'].astype(int)
df = df.merge(BufrCodes, how = 'outer', on = 'kx')
df = df[~df['bendingAngle'].isna()]
print(df)
print(df.columns)
##############################

#rootdir=df.filepath[0]
#df = df.sort_values(by=['DATE','kx'])

##############################
#saved_dataframe_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO/Y2009/M06/GPSRO.Y2009.M05.ioda_nc4.csv'
df.to_csv(saved_dataframe_path, index=False)
